In [24]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
years = pd.period_range(start='2003',end='2019',freq="Y")

In [71]:
for year in years:
    for tournament in range(1,4):
        r = requests.get("https://web2.acbl.org/tournaments/Results/NABC/NABC{}{}.HTM".format(str(year)[2:],tournament))
        print(r.url)
        

https://web2.acbl.org/tournaments/Results/NABC/NABC031.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC032.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC033.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC041.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC042.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC043.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC051.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC052.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC053.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC061.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC062.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC063.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC071.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC072.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC073.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC081.HTM
https://web2.acbl.org/tournaments/Results/NABC/NABC082.H

In [91]:
df = pd.read_html("https://web3.acbl.org/mpholdings")[0]
df.columns = ['level','this_range','this_range_+','this_range_-','this_range_+_pct','this_range_-_pct']
print(51940/sum(df.this_range))
df

0.3195993009919023


,level,this_range,this_range_+,this_range_-,this_range_+_pct,this_range_-_pct
0,"10,000+ (Grand LM)",383,383,162133,0.24,99.76
1,"10,000+ (Platinum LM)",492,875,161641,0.54,99.46
2,"9,000 - 10,000",202,1077,161439,0.67,99.33
3,"8,000 - 9,000",295,1372,161144,0.85,99.15
4,"7,500 - 8,000",184,1556,160960,0.96,99.04
5,"7,000 - 7,500",198,1754,160762,1.08,98.92
6,"6,000 - 7,000",609,2363,160153,1.46,98.54
7,"5,000 - 6,000",1050,3413,159103,2.10,97.90
8,"4,000 - 5,000",1634,5047,157469,3.11,96.89
9,"3,000 - 4,000",3065,8112,154404,5.00,95.00
